# Tools

This notebook contains all the tools that will be used by the agent.

In [2]:
import requests
import sqlite3
import pandas as pd

from smolagents import tool, Tool

from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

## Creating simple tool

A Python function must be annotated with `@tool`. It should also have a docstring describing what does the function do, what does it return and the description of its parameters.

### City to location 

The following function look up at latitude and longitude of a city.

In [3]:
# TODO: Load CSV file containing latitude, longitude and altitude of cities
# https://github.com/bahar/WorldCityLocations/tree/master
df = pd.read_csv('data/cities_latlng.csv', sep=';')
df.columns = [ 'id', 'country', 'city', 'latitude', 'longitude', 'altitude' ]
df.drop('country', axis=1, inplace=True)

In [4]:
# TODO: Explore the loaded dataframe
df.head()

,id,city,latitude,longitude,altitude
0,2,Kandahar,31.610000,65.699997,1015
1,3,Mazar-e Sharif,36.706944,67.112221,369
2,4,Herat,34.340000,62.189999,927
3,5,Jalalabad,34.420000,70.449997,573
4,6,Konduz,36.720000,68.860001,394


In [5]:
df[df['city'] == 'Singapore']

,id,city,latitude,longitude,altitude
10566,10568,Singapore,1.29027,103.851959,164


In [12]:
# TODO: Add tool description

# Add the @tool decorator to the function
@tool
def get_latlng(city: str) -> any:
   """
   Get the latitude, longitude and altitude of a city. The result is return in a dictionary

   Args:
       city: the name of the city that you want to get the latitude, longitude and altitude. The city name is case insensitive

   Returns:
      any: a dictionary with the following keys: city, latitude, longitude, altitude

   Example:
      result = get_latlng('singapore') # city name is case insensitive
   """
   r = df.query(f"city.str.lower() == '{city.lower()}'")
   return { 'city': city, 'latitude': r.iloc[0]['latitude'], 'longitude': r.iloc[0]['longitude'], 'altitude': r.iloc[0]['altitude'] }

In [13]:
# TODO: Test get_latlng method
# case insensitive search
get_latlng('kuala lumpur')

{'city': 'kuala lumpur',
 'latitude': np.float64(3.1666667),
 'longitude': np.float64(101.6999969),
 'altitude': np.int64(62)}

### Temperature at latitude and longitude

The following function lookup the weather at the given latitude and longtude.

In [17]:
# TODO: Add tool description

@tool
def get_temperature(latitude: float, longitude: float) -> any:
   """ 
   Get the temperature of a city given by its latitude and longitude. 

   Args:
      latitude: latitude of the city given in float
      longitude: longitude of the city given in float

   Return:
      any: a dictionary with the following keys: temperature_unit, temperature

   Example:
      result = get_temperature(3.1666667, 101.6999969)
   """
   url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m"
   resp = requests.get(url)
   j = resp.json()
   if resp.status_code >= 400:
      raise Exception(j['reason'])
   temperature = j['current']['temperature_2m']
   units = j['current_units']['temperature_2m']
   return { "temperature_unit": units, "temperature": temperature }

In [18]:
# TODO: Test get_temperature method
get_temperature(3.1666667, 101.6999969)

{'temperature_unit': '°C', 'temperature': 31.2}

In [19]:
# TODO: Create a web search tool

def web_search(query: str) -> any:

   search = DuckDuckGoSearchRun()
   return search.invoke(query)

In [21]:
%pip duckduckgo_search

ERROR: unknown command "duckduckgo_search"
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# TODO: Test the web search


### Query relational database

The following function queries a relational database (SQLite) view called `album_track`. The table's schema is as follows:
| Field name  | Type          |
|-------------|---------------| 
| AlbumId     | integer       |
| Title       | nvarchar(160) |
| track_name  | nvarchar(200) |
| artist_name | nvarchar(120) |
| duration    | integer       |
| composer    | nvarchar(220) |


In [ ]:
# TODO: Add tool description

@tool
def query_album_track(sql_query: str) -> any:
   """ 
   Perform SQL queries on the album_track table only. The result is return in a list.
   The table name is album_track and has the following columns:
      AlbumId: integer
      Title: nvarchar(160)
      track_name: nvarchar(200)
      artist_name: nvarchar(120)
      duration: integer
      composer: nvarchar(220)
   The duration column is in milliseconds. 
   The database is SQLlite.

   Args:
      sql_query: a valid SQL query or SQL dialect for SQLlite 

   Return: 
      any: list of tuple. Each element correspond to a record produced by the sql_query parameter

   Example:
      result = query_album_track('select * from album_track where artist_name like "%abba%"')

   """
   database = "data/chinook_sqlite.sqlite" 
   conn = sqlite3.connect(database)
   try:
      cursor = conn.cursor() 
      rows = cursor.execute(sql_query)
      return rows.fetchall()
   finally:
      conn.close()

In [26]:
# TODO: Test the query_album_track function
query_album_track('select * from album_track where artist_name like "%abba%"')

[(16, 'Black Sabbath', 'Black Sabbath', 'Black Sabbath', 382066, None),
 (16, 'Black Sabbath', 'The Wizard', 'Black Sabbath', 264829, None),
 (16,
  'Black Sabbath',
  'Behind The Wall Of Sleep',
  'Black Sabbath',
  217573,
  None),
 (16, 'Black Sabbath', 'N.I.B.', 'Black Sabbath', 368770, None),
 (16, 'Black Sabbath', 'Evil Woman', 'Black Sabbath', 204930, None),
 (16, 'Black Sabbath', 'Sleeping Village', 'Black Sabbath', 644571, None),
 (16, 'Black Sabbath', 'Warning', 'Black Sabbath', 212062, None),
 (17,
  'Black Sabbath Vol. 4 (Remaster)',
  'Wheels Of Confusion / The Straightener',
  'Black Sabbath',
  494524,
  'Tony Iommi, Bill Ward, Geezer Butler, Ozzy Osbourne'),
 (17,
  'Black Sabbath Vol. 4 (Remaster)',
  "Tomorrow's Dream",
  'Black Sabbath',
  192496,
  'Tony Iommi, Bill Ward, Geezer Butler, Ozzy Osbourne'),
 (17,
  'Black Sabbath Vol. 4 (Remaster)',
  'Changes',
  'Black Sabbath',
  286275,
  'Tony Iommi, Bill Ward, Geezer Butler, Ozzy Osbourne'),
 (17,
  'Black Sabbath

### Tools with states

The following isn an example of a more complex tool that requires initialisation

In [27]:
# extend Tool
class SQLiteTool(Tool):

   name = "chinook_sqlite"
   description = """ 
      Perform SQL queries on the album_track table only. The result is return in a list.
      The table name is album_track and has the following columns:
         AlbumId: integer
         Title: nvarchar(160)
         track_name: nvarchar(200)
         artist_name: nvarchar(120)
         duration: integer
         composer: nvarchar(220)
      The duration column is in milliseconds. 
      The database is SQLlite.
   """

   # Args:
   inputs = {
      'sql_query': {
         'type': 'string',
         'description': 'A valid SQL query or SQL dialect for SQLlite'
      }
   }

   output_type = "any"

   def __init__(self, db_file):
      self.db_file = db_file 
      # must call setup() - to initialize the tool
      self.setup()

   # tool logic is in forward()
   def forward(self, sql_query: str) -> any:
      conn = sqlite3.connect(self.db_file)
      try:
         cursor = conn.cursor() 
         rows = cursor.execute(sql_query)
         return rows.fetchall()
      finally:
         conn.close()   